# Augmentation des données

In [8]:
import warnings
warnings.simplefilter(action='ignore')
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
import imageio
import os

In [9]:
ia.seed(1)

def augment(image, image2, n_augment=8):

    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    seq = iaa.Sequential(
        [
            iaa.Fliplr(0.5),

            sometimes(iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                rotate=(-45, 45),
                shear=(-16, 16),
                order=[0, 1],
                cval=(0, 255),
                mode=ia.ALL
            )),

            iaa.SomeOf((0, 5),
                [
                    sometimes(
                        iaa.Superpixels(
                            p_replace=(0, 1.0),
                            n_segments=(20, 200)
                        )
                    ),

                    iaa.OneOf([
                        iaa.GaussianBlur((0, 3.0)),
                        iaa.AverageBlur(k=(2, 7)),
                        iaa.MedianBlur(k=(3, 11)),
                    ]),

                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),

                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),

                    sometimes(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0, 0.7)),
                        iaa.DirectedEdgeDetect(
                            alpha=(0, 0.7), direction=(0.0, 1.0)
                        ),
                    ])),

                    iaa.AdditiveGaussianNoise(
                        loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                    ),

                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.1), per_channel=0.5),
                        iaa.CoarseDropout(
                            (0.03, 0.15), size_percent=(0.02, 0.05),
                            per_channel=0.2
                        ),
                    ]),

                    iaa.Invert(0.05, per_channel=True), 

                    iaa.Add((-10, 10), per_channel=0.5),

                    iaa.Multiply((0.5, 1.5), per_channel=0.5),

                    iaa.LinearContrast((0.5, 2.0), per_channel=0.5),

                    iaa.Grayscale(alpha=(0.0, 1.0)),

                    sometimes(
                        iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
                    ),

                    sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05)))
                ],
                random_order=True
            )
        ],
        random_order=True
    )

    
    segmap = SegmentationMapsOnImage(image2, shape=image.shape)
    images_aug = []
    segmaps_aug = []
    for _ in range(n_augment):
        images_aug_i, segmaps_aug_i = seq(image=image, segmentation_maps=segmap)
        images_aug.append(images_aug_i)
        segmaps_aug.append(segmaps_aug_i.get_arr())
    
    return images_aug, segmaps_aug

In [10]:
original_imgs_path = "./data/leftImg8bit"

cities = {"train":[dirs for root, dirs, files in os.walk("/".join([original_imgs_path, "train"]))][0]}

paths = []
tgt_paths = []

for city in cities["train"]:
    tmp_path = "/".join([original_imgs_path, "train", city])
    lbl_tmp_path = tmp_path.replace("leftImg8bit", "gtFine")
    for _, _, files in os.walk(tmp_path):
        for name in files:
            if name.endswith(("_leftImg8bit.png")):
                paths += ["/".join([tmp_path, name])]
                tgt_paths += ["/".join([lbl_tmp_path, name]).replace("leftImg8bit", "gtFine_labelIds")]


In [14]:
for pth in range(len(paths)):
    img = imageio.imread(paths[pth])
    tgt = imageio.imread(tgt_paths[pth])
    img_t, tgt_t = augment(img, tgt)
    for n in range(len(img_t)):
        aug_img = img_t[n]
        aug_tgt = tgt_t[n]
        new_path = f"{paths[pth][:-4]}_augment_{n}.png"
        new_tgt_path = f"{tgt_paths[pth][:-4]}_augment_{n}.png"
        imageio.imwrite(new_path, aug_img)
        imageio.imwrite(new_tgt_path, aug_tgt)
    